# Deploy a training pipeline with Python function components

### Connect to your workspace

To connect to a workspace, we need identifier parameters - a subscription ID, resource group name, and workspace name. Since you're working with a compute instance, managed by Azure Machine Learning, you can use the default values to connect to the workspace.

In [ ]:
from dotenv import load_dotenv
import os
load_dotenv() # take environment variables from .env.


subscription = os.environ["SUBSCRIPTION"]
resource_group = os.environ["RESOURCE_GROUP"]
ws_name = os.environ["WORKSPACE_NAME"]
compute_cluster = os.environ["COMPUTE_CLUSTER"]

### Authenticate and get a handle to the workspace

Use the Azure ML SDK to authenticate and get a handle to the workspace using the provided subscription ID, resource group name, and workspace name.

In [ ]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

# authenticate
credential = DefaultAzureCredential()

# Get a handle to the workspace
ml_client = MLClient(
    credential=credential,
    subscription_id=subscription,
    resource_group_name=resource_group,
    workspace_name=ws_name,
)

### Create Data store and upload training data

Create a data store in the Azure ML workspace and upload the training data from the local directory. You only need to run this once. Once the data has been uploaded, you only have to run the cell after this one. 

In [ ]:
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes

data_path = "../data"
dataset_name = "diabetes-data-train"

patient_dataset = Data(
    path=data_path,
    type=AssetTypes.URI_FOLDER,
    description="Training data for diabetes prediction",
    name=dataset_name,
)
ml_client.data.create_or_update(patient_dataset)

### Load the dataset

Retrieve the uploaded dataset from the Azure ML workspace.

In [ ]:
patient_dataset = ml_client.data.get(
    name="diabetes-data-train", label="latest"
)

print(patient_dataset)

### Define pipeline components

Define the functions that will be used as components in the pipeline. We need to import the components from the python files first. Run the directory settings only once to set the project root root relative to the current notebook's location. Confirm the project_root is correct. 

In [ ]:
import os
from pathlib import Path

# Determine the project root relative to the current notebook's location
notebook_dir = Path.cwd()
project_root = notebook_dir.parent  # Adjust to go one level up to the project root

# Change the working directory to the project root
os.chdir(project_root)

print(project_root)

In [ ]:

from py_func_based_pipeline.components.compare_models import compare_two_models
from py_func_based_pipeline.components.fix_missing_data import remove_empty_rows
from py_func_based_pipeline.components.normalize_data import normalize_data
from py_func_based_pipeline.components.train_decision_tree import train_decision_tree_classifier_model
from py_func_based_pipeline.components.train_logistic_regression import train_logistic_regression_classifier_model


You can upload the components to AML.

In [ ]:
ml_client.components.create_or_update(remove_empty_rows)
ml_client.components.create_or_update(normalize_data)
ml_client.components.create_or_update(train_logistic_regression_classifier_model)
ml_client.components.create_or_update(train_decision_tree_classifier_model)
ml_client.components.create_or_update(compare_two_models)


In [ ]:
from azure.ai.ml import Input
from azure.ai.ml.constants import AssetTypes
from azure.ai.ml.dsl import pipeline

@pipeline()
def diabetes_classification_pipeline_component( pipeline_job_input: Input):
    clean_data = remove_empty_rows(input_data=pipeline_job_input)
    normalized_data = normalize_data(input_data=clean_data.outputs.output_data)
    train_model_decision_tree = train_decision_tree_classifier_model(training_data=normalized_data.outputs.output_data)
    train_model_logistic_regression = train_logistic_regression_classifier_model(training_data=normalized_data.outputs.output_data)
    better_model = compare_two_models(model1=train_model_decision_tree.outputs.model_output_decision_tree, 
                                            model1_metrics=train_model_decision_tree.outputs.metrics_output,
                                            model2=train_model_logistic_regression.outputs.model_output_logistic_reg,
                                            model2_metrics=train_model_logistic_regression.outputs.metrics_output)                             
    
    return {
        "pipeline_job_transformed_data": normalized_data.outputs.output_data,
        "pipeline_job_trained_model_decision_tree": train_model_decision_tree.outputs.model_output_decision_tree,
        "pipeline_job_trained_model_logistic_regression": train_model_logistic_regression.outputs.model_output_logistic_reg,
        "pipeline_job_better_model": better_model.outputs.better_model,	
    }


Register the pipeline component to the workspace:

In [ ]:
registered_train_pipeline_component = ml_client.components.create_or_update(
    diabetes_classification_pipeline_component
)

Use the registered pipeline component to create a pipeline job:

In [ ]:
pipeline_job = registered_train_pipeline_component(pipeline_job_input=Input(type=AssetTypes.URI_FILE, path=patient_dataset.path))
print(pipeline_job)

In [ ]:
# set pipeline level compute
pipeline_job.settings.default_compute = compute_cluster
# set pipeline level datastore
pipeline_job.settings.default_datastore = "workspaceblobstore"

# submit job to workspace
pipeline_job = ml_client.jobs.create_or_update(
    pipeline_job, experiment_name="pipeline_diabetes_training_py_func"
)
pipeline_job